In [ ]:
int calcularRandom ();

int main () {

    pid_t procId = fork ();

    if ( procId == 0 ) {

        string archivo ( "main1.cc" );
        MemoriaCompartida<int> memoria;
        int estadoMemoria = memoria.crear ( archivo,'R' );

        cout << "Hijo: duermo 5 segundos..." << endl;
        sleep ( 5 );

* Es proceso hijo por el pid
* Obtenemos un objeto string de contenido "main.cc". Para que? Por que?
* Que corno es el `<int>`?
* No entendi, que hace el crear?
* Eu, ahi hay un sleep

In [ ]:
        if ( estadoMemoria == SHM_OK ) {
            int resultado = memoria.leer ();
            cout << "Hijo: leo el numero " << resultado << " de la memoria compartida" << endl;
            memoria.liberar ();
        } else {
            cout << "Hijo: error en memoria compartida: " << estadoMemoria << endl;
        }
        cout << "Hijo: fin del proceso" << endl;
        exit ( 0 );

* Ah, manejo de errores
* Leer y mostrar, todo bien

In [ ]:
    } else {

        string archivo ( "main1.cc" );
        MemoriaCompartida<int> memoria;
        int estadoMemoria = memoria.crear ( archivo,'R' );
        if ( estadoMemoria == SHM_OK ) {

            // escribe un dato para el hijo
            int random = calcularRandom ();
            cout << "Padre: escribo el numero " << random << " en la memoria compartida" << endl;
            memoria.escribir ( random );

            // espera a que termine el hijo
            wait ( NULL );
            // libera la memoria
            memoria.liberar ();
        } else {
            cout << "Padre: error en memoria compartida: " << estadoMemoria << endl;
        }

        cout << "Padre: fin del proceso" << endl;
        exit ( 0 );

    }

* Libere todo, todo legal
* Nada distinto o raro de lo que paso con el hijo

In [1]:
make ej1
./ejemplo

g++ main1.cc -std=c++11 -D EJEMPLO_1 -o ejemplo
Padre: escribo el numero 9 en la memoria compartida
Hijo: duermo 5 segundos...
Hijo: leo el numero 9 de la memoria compartida
Hijo: fin del proceso
Padre: fin del proceso


In [2]:
ipcs


------ Message Queues --------
key        msqid      owner      perms      used-bytes   messages    

------ Shared Memory Segments --------
key        shmid      owner      perms      bytes      nattch     status      

------ Semaphore Arrays --------
key        semid      owner      perms      nsems     



In [ ]:
#ifndef MEMORIACOMPARTIDA_H_
#define MEMORIACOMPARTIDA_H_

#define SHM_OK  0
#define ERROR_FTOK -1
#define ERROR_SHMGET -2
#define ERROR_SHMAT -3

#include <sys/types.h>
#include <sys/ipc.h>
#include <sys/shm.h>
#include <string>


template <class T> class MemoriaCompartida {

private:
    int shmId;
    T*    ptrDatos;

    int cantidadProcesosAdosados () const;

public:
    MemoriaCompartida ();
    ~MemoriaCompartida ();
    int crear ( const std::string& archivo,const char letra );
    void liberar ();
    void escribir ( const T& dato );
    T leer () const;

};


`const` after a function declaration means that the function is not allowed to change any class members (except ones that are marked `mutable`). So this use of const only makes sense, and is hence only allowed, for member functions.

In [ ]:
template <class T> MemoriaCompartida<T> :: MemoriaCompartida() : shmId(0), ptrDatos(NULL) {
}

In [ ]:
template <class T> int MemoriaCompartida<T> :: crear ( const std::string& archivo,const char letra ) {

    // generacion de la clave
    key_t clave = ftok ( archivo.c_str(),letra );
    if ( clave == -1 )
        return ERROR_FTOK;
    else {
        // creacion de la memoria compartida
        this->shmId = shmget ( clave,sizeof(T),0644|IPC_CREAT );

        if ( this->shmId == -1 )
            return ERROR_SHMGET;
        else {
            // attach del bloque de memoria al espacio de direcciones del proceso
            void* ptrTemporal = shmat ( this->shmId,NULL,0 );

            if ( ptrTemporal == (void *) -1 ) {
                return ERROR_SHMAT;
            } else {
                this->ptrDatos = static_cast<T*> (ptrTemporal);
                return SHM_OK;
            }
        }
    }
}

In [ ]:
template <class T> void MemoriaCompartida<T> :: liberar () {
    // detach del bloque de memoria
    shmdt ( static_cast<void*> (this->ptrDatos) );

    int procAdosados = this->cantidadProcesosAdosados ();

    if ( procAdosados == 0 ) {
        shmctl ( this->shmId,IPC_RMID,NULL );
    }
}

template <class T> void MemoriaCompartida<T> :: escribir ( const T& dato ) {
    * (this->ptrDatos) = dato;
}

template <class T> T MemoriaCompartida<T> :: leer () const {
    return ( *(this->ptrDatos) );
}

template <class T> int MemoriaCompartida<T> :: cantidadProcesosAdosados () const {
    shmid_ds estado;
    shmctl ( this->shmId,IPC_STAT,&estado );
    return estado.shm_nattch;
}
